In [ ]:
!pip install d2l

In [ ]:

import math
import random
import torch
import time
from torch import nn
from torch.optim.optimizer import Optimizer
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from d2l import torch as d2l
from collections import deque
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Read data
df_spark = pd.read_csv('df_spark.csv')
df_spark = df_spark.drop(columns="Unnamed: 0")

y = df_spark.iloc[:, 0].values
X = df_spark.iloc[:, 1:].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)

# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the custom optimizer
class FCSGD_G_L(Optimizer):
    def __init__(self, params, lr=1e-1, weight_decay=1e-8, r=1.1):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= weight_decay:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        if not -2.0 <= r:
            raise ValueError("Invalid r value: {}".format(r))
        defaults = dict(lr=lr, weight_decay=weight_decay, r=r)
        super(FCSGD_G_L, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(FCSGD_G_L, self).__setstate__(state)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('FCSGD_G_L does not support sparse gradients')

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    state['present_grad'] = torch.zeros_like(p.data)
                    # Previous gradient
                    state['previous_grad_1'] = torch.zeros_like(p.data)
                    state['previous_grad_2'] = torch.zeros_like(p.data)
                    state['previous_grad_3'] = torch.zeros_like(p.data)
                    state['previous_grad_4'] = torch.zeros_like(p.data)
                    state['previous_grad_5'] = torch.zeros_like(p.data)
                    state['previous_grad_6'] = torch.zeros_like(p.data)
                    state['previous_grad_7'] = torch.zeros_like(p.data)
                    state['previous_grad_8'] = torch.zeros_like(p.data)
                    state['previous_grad_9'] = torch.zeros_like(p.data)
                    state['previous_grad_10'] = torch.zeros_like(p.data)
                    # Fractional order
                    r = group['r']
                    # Coefficients by a novel G-L
                    state['w0'] = 1
                    state['w1'] = (1 - (r + 1) / 2) * state['w0']
                    state['w2'] = (1 - (r + 1) / 3) * state['w1']
                    state['w3'] = (1 - (r + 1) / 4) * state['w2']
                    state['w4'] = (1 - (r + 1) / 5) * state['w3']
                    state['w5'] = (1 - (r + 1) / 6) * state['w4']
                    state['w6'] = (1 - (r + 1) / 7) * state['w5']
                    state['w7'] = (1 - (r + 1) / 8) * state['w6']
                    state['w8'] = (1 - (r + 1) / 9) * state['w7']
                    state['w9'] = (1 - (r + 1) / 10) * state['w8']
                    state['w10'] = (1 - (r + 1) / 11) * state['w9']

                fractional_order_grad = state['present_grad']

                a = [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
                random.shuffle(a)

                fractional_order_grad.add_(1, grad.clone())\
                    .add_(a[0] * state['w1'], state['previous_grad_1'])\
                    .add_(a[1] * state['w2'], state['previous_grad_2'])\
                    .add_(a[2] * state['w3'], state['previous_grad_3'])\
                    .add_(a[3] * state['w4'], state['previous_grad_4'])\
                    .add_(a[4] * state['w5'], state['previous_grad_5'])\
                    .add_(a[5] * state['w6'], state['previous_grad_6'])\
                    .add_(a[6] * state['w7'], state['previous_grad_7'])\
                    .add_(a[7] * state['w8'], state['previous_grad_8'])\
                    .add_(a[8] * state['w9'], state['previous_grad_9'])\
                    .add_(a[9] * state['w10'], state['previous_grad_10'])

                state['previous_grad_10'] = state['previous_grad_9']
                state['previous_grad_9'] = state['previous_grad_8']
                state['previous_grad_8'] = state['previous_grad_7']
                state['previous_grad_7'] = state['previous_grad_6']
                state['previous_grad_6'] = state['previous_grad_5']
                state['previous_grad_5'] = state['previous_grad_4']
                state['previous_grad_4'] = state['previous_grad_3']
                state['previous_grad_3'] = state['previous_grad_2']
                state['previous_grad_2'] = state['previous_grad_1']
                state['previous_grad_1'] = grad.clone()

                state['step'] += 1

                if group['weight_decay'] != 0:
                    fractional_order_grad.add_(group['weight_decay'], p.data)

                fractional_order_grad_1 = fractional_order_grad
                state['present_grad'] = torch.zeros_like(p.data)
                step_size = group['lr']
                p.data.add_(-step_size, fractional_order_grad_1)

        return loss

# Define the PyTorch model
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

# Initialize model, optimizer, and loss function
input_size = X_train.shape[1]
hidden_size = 50
output_size = len(np.unique(y))
model = SimpleMLP(input_size, hidden_size, output_size)

optimizer = FCSGD_G_L(model.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    train_outputs = model(X_train_tensor)
    test_outputs = model(X_test_tensor)
    train_preds = torch.argmax(train_outputs, dim=1)
    test_preds = torch.argmax(test_outputs, dim=1)
    train_acc = (train_preds == y_train_tensor).float().mean()
    test_acc = (test_preds == y_test_tensor).float().mean()

print(f'Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}')

# Classification report
from sklearn.metrics import classification_report, confusion_matrix

y_train_pred = train_preds.numpy()
y_test_pred = test_preds.numpy()
target_names = ['Normal', 'DoSattack', 'scan', 'malitiousControl', 'malitiousOperation', 'spying', 'dataProbing', 'wrongSetUp']

print(classification_report(y_train, y_train_pred, target_names=target_names))
print(classification_report(y_test, y_test_pred, target_names=target_names))

# Confusion matrix
cnf_matrix = confusion_matrix(y_test, y_test_pred)
print(cnf_matrix)


In [ ]:
#for time
import math

import random
import torch
import time
from torch import nn
from torch.optim import Adam, SGD
from torch.optim.optimizer import Optimizer
from sklearn.metrics import r2_score, classification_report, confusion_matrix
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from d2l import torch as d2l
from collections import deque
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Read data
df_spark = pd.read_csv('df_spark.csv')
df_spark = df_spark.drop(columns="Unnamed: 0")

y = df_spark.iloc[:, 0].values
X = df_spark.iloc[:, 1:].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the custom optimizer
class FCSGD_G_L(Optimizer):
    def __init__(self, params, lr=1e-1, weight_decay=1e-8, r=0.8):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= weight_decay:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        if not -2.0 <= r:
            raise ValueError("Invalid r value: {}".format(r))
        defaults = dict(lr=lr, weight_decay=weight_decay, r=r)
        super(FCSGD_G_L, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(FCSGD_G_L, self).__setstate__(state)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('FCSGD_G_L does not support sparse gradients')

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    state['present_grad'] = torch.zeros_like(p.data)
                    # Previous gradient
                    state['previous_grad_1'] = torch.zeros_like(p.data)
                    state['previous_grad_2'] = torch.zeros_like(p.data)
                    state['previous_grad_3'] = torch.zeros_like(p.data)
                    state['previous_grad_4'] = torch.zeros_like(p.data)
                    state['previous_grad_5'] = torch.zeros_like(p.data)
                    state['previous_grad_6'] = torch.zeros_like(p.data)
                    state['previous_grad_7'] = torch.zeros_like(p.data)
                    state['previous_grad_8'] = torch.zeros_like(p.data)
                    state['previous_grad_9'] = torch.zeros_like(p.data)
                    state['previous_grad_10'] = torch.zeros_like(p.data)
                    # Fractional order
                    r = group['r']
                    # Coefficients by a novel G-L
                    state['w0'] = 1
                    state['w1'] = (1 - (r + 1) / 2) * state['w0']
                    state['w2'] = (1 - (r + 1) / 3) * state['w1']
                    state['w3'] = (1 - (r + 1) / 4) * state['w2']
                    state['w4'] = (1 - (r + 1) / 5) * state['w3']
                    state['w5'] = (1 - (r + 1) / 6) * state['w4']
                    state['w6'] = (1 - (r + 1) / 7) * state['w5']
                    state['w7'] = (1 - (r + 1) / 8) * state['w6']
                    state['w8'] = (1 - (r + 1) / 9) * state['w7']
                    state['w9'] = (1 - (r + 1) / 10) * state['w8']
                    state['w10'] = (1 - (r + 1) / 11) * state['w9']

                fractional_order_grad = state['present_grad']

                a = [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
                random.shuffle(a)

                fractional_order_grad.add_(1, grad.clone())\
                    .add_(a[0] * state['w1'], state['previous_grad_1'])\
                    .add_(a[1] * state['w2'], state['previous_grad_2'])\
                    .add_(a[2] * state['w3'], state['previous_grad_3'])\
                    .add_(a[3] * state['w4'], state['previous_grad_4'])\
                    .add_(a[4] * state['w5'], state['previous_grad_5'])\
                    .add_(a[5] * state['w6'], state['previous_grad_6'])\
                    .add_(a[6] * state['w7'], state['previous_grad_7'])\
                    .add_(a[7] * state['w8'], state['previous_grad_8'])\
                    .add_(a[8] * state['w9'], state['previous_grad_9'])\
                    .add_(a[9] * state['w10'], state['previous_grad_10'])

                state['previous_grad_10'] = state['previous_grad_9']
                state['previous_grad_9'] = state['previous_grad_8']
                state['previous_grad_8'] = state['previous_grad_7']
                state['previous_grad_7'] = state['previous_grad_6']
                state['previous_grad_6'] = state['previous_grad_5']
                state['previous_grad_5'] = state['previous_grad_4']
                state['previous_grad_4'] = state['previous_grad_3']
                state['previous_grad_3'] = state['previous_grad_2']
                state['previous_grad_2'] = state['previous_grad_1']
                state['previous_grad_1'] = grad.clone()

                state['step'] += 1

                if group['weight_decay'] != 0:
                    fractional_order_grad.add_(group['weight_decay'], p.data)

                fractional_order_grad_1 = fractional_order_grad
                state['present_grad'] = torch.zeros_like(p.data)
                step_size = group['lr']
                p.data.add_(-step_size, fractional_order_grad_1)

        return loss

# Define the PyTorch model
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

# Initialize model, optimizer, and loss function
input_size = X_train.shape[1]
hidden_size = 50
output_size = len(np.unique(y))
model = SimpleMLP(input_size, hidden_size, output_size)

# Function to measure training time for an optimizer
def measure_training_time(optimizer_class, **optimizer_kwargs):
    model = SimpleMLP(input_size, hidden_size, output_size)
    optimizer = optimizer_class(model.parameters(), **optimizer_kwargs)
    criterion = nn.CrossEntropyLoss()

    start_time = time.time()
    for epoch in range(1):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    end_time = time.time()
    return end_time - start_time

# Measure average training time per epoch
num_repeats = 5

fcsdg_l_times = [measure_training_time(FCSGD_G_L, lr=0.1) for _ in range(num_repeats)]
adam_times = [measure_training_time(Adam, lr=0.001) for _ in range(num_repeats)]
sgd_times = [measure_training_time(SGD, lr=0.01) for _ in range(num_repeats)]

avg_fcsdg_l_time = np.mean(fcsdg_l_times)
avg_adam_time = np.mean(adam_times)
avg_sgd_time = np.mean(sgd_times)

print(f'Average training time per epoch (FCSGD_G_L): {avg_fcsdg_l_time:.4f} seconds')
print(f'Average training time per epoch (Adam): {avg_adam_time:.4f} seconds')
print(f'Average training time per epoch (SGD): {avg_sgd_time:.4f} seconds')

# Training loop with timing
num_epochs = 20
optimizer_fcsdg_l = FCSGD_G_L(model.parameters(), lr=0.1)
optimizer_adam = Adam(model.parameters(), lr=0.001)
optimizer_sgd = SGD(model.parameters(), lr=0.01)

def train_model(optimizer, model, num_epochs):
    model.train()
    criterion = nn.CrossEntropyLoss()
    epoch_times = []
    for epoch in range(num_epochs):
        start_time = time.time()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
        end_time = time.time()
        epoch_times.append(end_time - start_time)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    return epoch_times

# Train models and measure times
fcsdg_l_times = train_model(optimizer_fcsdg_l, model, num_epochs)
adam_times = train_model(optimizer_adam, model, num_epochs)
sgd_times = train_model(optimizer_sgd, model, num_epochs)

# Average time per epoch
avg_fcsdg_l_time = np.mean(fcsdg_l_times)
avg_adam_time = np.mean(adam_times)
avg_sgd_time = np.mean(sgd_times)

print(f'Average training time per epoch (FCSGD_G_L): {avg_fcsdg_l_time:.4f} seconds')
print(f'Average training time per epoch (Adam): {avg_adam_time:.4f} seconds')
print(f'Average training time per epoch (SGD): {avg_sgd_time:.4f} seconds')

# Evaluate the model
def evaluate_model(model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor):
    model.eval()
    with torch.no_grad():
        train_outputs = model(X_train_tensor)
        test_outputs = model(X_test_tensor)
        train_preds = torch.argmax(train_outputs, dim=1)
        test_preds = torch.argmax(test_outputs, dim=1)
        train_acc = (train_preds == y_train_tensor).float().mean()
        test_acc = (test_preds == y_test_tensor).float().mean()

    print(f'Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}')
    return train_preds, test_preds

train_preds, test_preds = evaluate_model(model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

# Classification report
y_train_pred = train_preds.numpy()
y_test_pred = test_preds.numpy()
target_names = ['Normal', 'DoSattack', 'scan', 'malitiousControl', 'malitiousOperation', 'spying', 'dataProbing', 'wrongSetUp']

print(classification_report(y_train, y_train_pred, target_names=target_names))
print(classification_report(y_test, y_test_pred, target_names=target_names))

# Confusion matrix
cnf_matrix = confusion_matrix(y_test, y_test_pred)
print(cnf_matrix)
